<a href="https://colab.research.google.com/github/dbwofla11/DaconBaseLine_LLM-gemma-7b-/blob/master/TransFormer_Base1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install --upgrade transformers


In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline

In [ ]:
# prompt: 구글 드라이브 마운팅

from google.colab import drive
drive.mount('/content/drive')


In [ ]:


# 데이터 경로 설정 (DaconData 폴더 내부의 파일명으로 수정)
train_path = '/content/drive/MyDrive/Colab Notebooks/DaconData/train.csv'
test_path = '/content/drive/MyDrive/Colab Notebooks/DaconData/test.csv'


try:
    train = pd.read_csv(train_path)
    test = pd.read_csv(test_path)
    print("Train data shape:", train.shape)
    print("Test data shape:", test.shape)

except FileNotFoundError:
    print(f"Error: Train or Test file not found at specified paths.")
    print(f"Train Path: {train_path}")
    print(f"Test Path: {test_path}")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_id = 'beomi/gemma-ko-7b'  # 모델 ID
bnb_config = None  # 사용 중인 설정

try:
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        quantization_config=bnb_config,
        device_map="auto",  # 자동으로 장치 맵핑
    )
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    print("모델이 성공적으로 로드되었습니다!")
except KeyError as e:
    print(f"KeyError 발생: {e}")
    print("모델 ID 또는 클래스 호환성을 확인하세요.")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00006.safetensors:   0%|          | 0.00/2.93G [00:00<?, ?B/s]

model-00002-of-00006.safetensors:   0%|          | 0.00/2.92G [00:00<?, ?B/s]

model-00003-of-00006.safetensors:   0%|          | 0.00/2.99G [00:00<?, ?B/s]

In [ ]:
def replace_words(input_text, match_dict):
    words = input_text.split()
    replaced_words = [match_dict.get(word, word) for word in words]
    return " ".join(replaced_words)

converted_reviews = test['input'].apply(lambda x: replace_words(x, match_dict)).tolist()

In [ ]:
submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DaconData/sample_submission.csv', encoding = 'utf-8-sig')
submission['output'] = converted_reviews
submission.to_csv('./baseline_submission.csv', index = False, encoding = 'utf-8-sig')

